In [20]:
%pip install gurobipy

import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# List of activities and vendors

Activities = ['A1','A2','A3','A4','A5']
Vendors = ['V1', 'V2', 'IH']

# Create dictionary for cost savings (Mn $)

a2v, savings = gp.multidict({
    ('A1', 'V1'): 15,
    ('A1', 'V2'): 25,
    ('A1', 'IH'): 0,
    ('A2', 'V1'): 20,
    ('A2', 'V2'): 16,
    ('A2', 'IH'): 0,
    ('A3', 'V1'): 11,
    ('A3', 'V2'): 17,
    ('A3', 'IH'): 0,
    ('A4', 'V1'): 22,
    ('A4', 'V2'): 18,
    ('A4', 'IH'): 0,
    ('A5', 'V1'): 15,
    ('A5', 'V2'): 35,
    ('A5', 'IH'): 0
})

# Create a dictionary to capture the annual coordination costs -in thousands of dollars from relocation.

ava2v2, coordinationCost = gp.multidict({
    ('A1','IH','A2','V1'): 13,
    ('A1','IH','A2','V2'): 9,
    ('A1','IH','A2','IH'): 1,
    ('A1','IH','A3','V1'): 15,
    ('A1','IH','A3','V2'): 10,
    ('A1','IH','A3','IH'): 1,
    ('A1','IH','A4','V1'): 22,
    ('A1','IH','A4','V2'): 8,
    ('A1','IH','A4','IH'): 1,
    ('A1','IH','A5','V1'): 10,
    ('A1','IH','A5','V2'): 17,
    ('A1','IH','A5','IH'): 1,

    ('A2','IH','A3','V1'): 17,
    ('A2','IH','A3','V2'): 23,
    ('A2','IH','A3','IH'): 1,
    ('A2','IH','A4','V1'): 14,
    ('A2','IH','A4','V2'): 22,
    ('A2','IH','A4','IH'): 1,
    ('A2','IH','A5','V1'): 21,
    ('A2','IH','A5','V2'): 11,
    ('A2','IH','A5','IH'): 1,

    ('A3','IH','A4','V1'): 18,
    ('A3','IH','A4','V2'): 22,
    ('A3','IH','A4','IH'): 1,
    ('A3','IH','A5','V1'): 17,
    ('A3','IH','A5','V2'): 8,
    ('A3','IH','A5','IH'): 1,

    ('A4','IH','A5','V1'): 17,
    ('A4','IH','A5','V2'): 16,
    ('A4','IH','A5','IH'): 1,

    ('A1','V1','A2','V1'): 1,
    ('A1','V1','A2','V2'): 19,
    ('A1','V1','A2','IH'): 21,
    ('A1','V1','A3','V1'): 1,
    ('A1','V1','A3','V2'): 26,
    ('A1','V1','A3','IH'): 31,
    ('A1','V1','A4','V1'): 1,
    ('A1','V1','A4','V2'): 11,
    ('A1','V1','A4','IH'): 17,
    ('A1','V1','A5','V1'): 1,
    ('A1','V1','A5','V2'): 18,
    ('A1','V1','A5','IH'): 9,

    ('A2','V1','A3','V1'): 1,
    ('A2','V1','A3','V2'): 21,
    ('A2','V1','A3','IH'): 30,
    ('A2','V1','A4','V1'): 1,
    ('A2','V1','A4','V2'): 20,
    ('A2','V1','A4','IH'): 10,
    ('A2','V1','A5','V1'): 1,
    ('A2','V1','A5','V2'): 25,
    ('A2','V1','A5','IH'): 15,

    ('A3','V1','A4','V1'): 1,
    ('A3','V1','A4','V2'): 28,
    ('A3','V1','A4','IH'): 14,
    ('A3','V1','A5','V1'): 1,
    ('A3','V1','A5','V2'): 15,
    ('A3','V1','A5','IH'): 13,

    ('A4','V1','A5','V1'): 1,
    ('A4','V1','A5','V2'): 25,
    ('A4','V1','A5','IH'): 27,

    ('A1','V2','A2','V1'): 11,
    ('A1','V2','A2','V2'): 1,
    ('A1','V2','A2','IH'): 15,
    ('A1','V2','A3','V1'): 20,
    ('A1','V2','A3','V2'): 1,
    ('A1','V2','A3','IH'): 10,
    ('A1','V2','A4','V1'): 9,
    ('A1','V2','A4','V2'): 1,
    ('A1','V2','A4','IH'): 11,
    ('A1','V2','A5','V1'): 12,
    ('A1','V2','A5','V2'): 1,
    ('A1','V2','A5','IH'): 8,

    ('A2','V2','A3','V1'): 7,
    ('A2','V2','A3','V2'): 1,
    ('A2','V2','A3','IH'): 11,
    ('A2','V2','A4','V1'): 15,
    ('A2','V2','A4','V2'): 1,
    ('A2','V2','A4','IH'): 18,
    ('A2','V2','A5','V1'): 14,
    ('A2','V2','A5','V2'): 1,
    ('A2','V2','A5','IH'): 20,

    ('A3','V2','A4','V1'): 22,
    ('A3','V2','A4','V2'): 1,
    ('A3','V2','A4','IH'): 22,
    ('A3','V2','A5','V1'): 20,
    ('A3','V2','A5','V2'): 1,
    ('A3','V2','A5','IH'): 10,

    ('A4','V2','A5','V1'): 25,
    ('A4','V2','A5','V2'): 1,
    ('A4','V2','A5','IH'): 15

})
 

model = gp.Model('outsourcing')

# Set global parameters 
model.params.nonConvex = 2

#source activity a to vendor v
source = model.addVars(a2v, vtype=GRB.BINARY, name="source")


# Activity sourcing constraint

activity_sourcing = model.addConstrs((gp.quicksum(source[a,v] for v in Vendors) == 1 for a in Activities), 
                                    name='activity_sourcing')


# Limit on number of activities

activities_limit = model.addConstrs((gp.quicksum(source[a,v] for a in Activities) <= 4 for v in Vendors), 
                                    name='activities_limit')


model.setObjective((gp.quicksum(savings[a,v]*source[a,v] for a,v in a2v) 
                    - gp.quicksum(coordinationCost[a,v,a2,v2]*source[a,v]*source[a2,v2] for a,v,a2,v2 in ava2v2) ),
                   GRB.MAXIMIZE)



# Verify model formulation

model.write('outsourcingQA.lp')

# Run optimization engine

model.optimize()


sourcing_plan = pd.DataFrame(columns=["Activities", "Vendors"])

count = 0

for v in Vendors:
    for a in Activities:
        if(source[a,v].x > 0.5):
            count += 1
            sourcing_plan = sourcing_plan.append({"Activities": a, "Vendors": v }, ignore_index=True )
sourcing_plan.index=['']*count
sourcing_plan


print("\n\n_________________________________________________________________________________")
print(f"Savings report")
print("_________________________________________________________________________________")
total_savings = 0
for v in Vendors:
    for a in Activities:
        if(source[a,v].x > 0.5):
            total_savings += 1000000*savings[a,v]

dollars_savings = '${:,.2f}'.format(total_savings)
print(f"The yearly total savings are {dollars_savings} dollars")

total_coordination_cost = 0
for a,v,a2,v2 in ava2v2:
    if(source[a,v].x*source[a2,v2].x > 0.5):
        total_coordination_cost += 1000000*coordinationCost[a,v,a2,v2]

dollars_coordination_cost = '${:,.2f}'.format(total_coordination_cost)
print(f"The yearly total coordination cost is {dollars_coordination_cost} dollars")

total_gross_margin = total_savings - total_coordination_cost
dollars_gross_margin = '${:,.2f}'.format(total_gross_margin)
print(f"The yearly total gross margin is {dollars_gross_margin} dollars")




Note: you may need to restart the kernel to use updated packages.
Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 8 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x99598468
Model has 90 quadratic objective terms
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+01]
  QObjective range [2e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective -54.0000000
Presolve time: 0.00s
Presolved: 98 rows, 105 columns, 300 nonzeros
Variable types: 0 continuous, 105 integer (105 binary)

Root relaxation: objective -9.940000e+01, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 